In [ ]:
from csv import *
from db_handler import *
from api_call_handler import *
import json

In [ ]:
keys = json.load(open('keys.json'))
API_KEY = keys['API_KEY']

In [ ]:
endpoint = 'sets'
new_df = get_df_from_api(endpoint,API_KEY,'20')

In [ ]:
create_table_statement(new_df, endpoint)

In [ ]:


replacements = {
    'timedelta64[ns]': 'varchar',
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp'
}
column_string = ''
for column, dtype in zip(new_df.columns, new_df.dtypes.replace(replacements)):
    column_string += '{} {},'.format(column, dtype)

table_statement = 'create table if not exists {} ({})'.format(
    endpoint, column_string)

table_statement

In [ ]:
load_dir = "initial_load"

In [ ]:
connection_data = json.load(open('connection_data.json'))
host_name = connection_data['host_name']
dbname = connection_data['dbname']
username = connection_data['username']
password = connection_data['password']
port = connection_data['port']
conn = None

conn = connect_to_db(host_name, dbname, username, password, port)

In [ ]:
curr = conn.cursor()

In [ ]:
create_table(curr)

In [ ]:
new_set_df = update_db(curr, df)

In [ ]:
append_from_df_to_db(curr,new_set_df)

In [ ]:
conn.commit()